In [8]:
import numpy as np
from sklearn.model_selection import train_test_split
import os

# Define directories
processed_data_dir = r'C:\Users\ACER\gitClones\QuickDrawGame\model_training\data\processed_data'

# Load image and label files
def load_data():
    images = []
    labels = []
    
    # Loop through all categories (replace with your actual labels)
    labels_list = ['airplane','bicycle', 'bus', 'car', 'car', 'computer', 'dog', 'elephant', 'fish', 'flower', 'horse', 'house', 'moon', 'rabbit', 'smiley face', 'star', 'sun', 'tree', 'truck']
    
    for label in labels_list:
        # Load the corresponding images and labels for each batch
        for batch_index in range(1, 11):  # Assuming 10 batches per label
            # Correct file paths for images and one-hot labels
            image_file = os.path.join(processed_data_dir, f'full_raw_{label}_images_{batch_index}.npy')
            label_file = os.path.join(processed_data_dir, f'full_raw_{label}_one_hot_labels_{batch_index}.npy')
            
            # Load images and one-hot encoded labels
            images_batch = np.load(image_file)
            labels_batch = np.load(label_file)
            
            images.append(images_batch)
            labels.append(labels_batch)
    
    # Concatenate all batches of images and labels
    images = np.concatenate(images, axis=0)
    labels = np.concatenate(labels, axis=0)
    
    # Reshape the images to ensure they're in the correct format (e.g., 28x28x1)
    images = images.reshape(-1, 28, 28, 1)  # Assuming 28x28 grayscale images
    
    return images, labels

# Load the data
images, labels = load_data()

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.2, random_state=42)

print(f"Training data shape: {X_train.shape}")
print(f"Validation data shape: {X_val.shape}")


Training data shape: (15200, 28, 28, 1)
Validation data shape: (3800, 28, 28, 1)


In [9]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers

In [10]:
from tensorflow.keras.backend import clear_session

clear_session()

In [11]:
# Load the MobileNetV2 model pre-trained on ImageNet
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(128, 128, 3))  # Use 128x128 or 224x224

# Freeze the base model to keep the pre-trained weights
for layer in base_model.layers[-4:]:  # Unfreeze the last 4 layers
    layer.trainable = True

# Create the model
model = Sequential([
    # Add the pre-trained base model
    base_model,
    
    # Global average pooling layer to reduce dimensionality
    GlobalAveragePooling2D(),
    
    # Add a dense layer for custom output
    Dense(128, activation='relu'),
    
    # Dropout layer to avoid overfitting
    Dropout(0.5),
    
    # Output layer with 19 classes (your number of categories)
    Dense(19, activation='softmax')  # 19 categories, softmax for multi-class classification
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

# Print model summary
model.summary()

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_128            │ (None, 4, 4, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       163,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 19)             │         2,451 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,424,403 (9.25 MB)

 Trainable params: 2,390,291 (9.12 MB)

 Non-trainable params: 34,112 (133.25 KB)

In [12]:
import cv2
import numpy as np

# Resize images to 32x32 and convert grayscale to RGB (3 channels)
def preprocess_images(images, image_size=128):  # Increase the image size to 128x128
    images_resized = []
    for img in images:
        img_resized = cv2.resize(img, (image_size, image_size))  # Resize to (128, 128)
        
        # If the image is grayscale (1 channel), convert it to RGB (3 channels)
        if img_resized.ndim == 2:  # If the image is grayscale (only 2 dimensions: height and width)
            img_resized = cv2.cvtColor(img_resized, cv2.COLOR_GRAY2RGB)  # Convert to RGB
        
        # If the image has only 1 channel (grayscale), convert it to RGB
        elif img_resized.shape[-1] == 1:  # Check if the image has only 1 channel (grayscale)
            img_resized = cv2.cvtColor(img_resized, cv2.COLOR_GRAY2RGB)  # Convert to RGB
        
        images_resized.append(img_resized)
    
    return np.array(images_resized)

# Assuming X_train and X_val are your training and validation image datasets
X_train = preprocess_images(X_train)
X_val = preprocess_images(X_val)

print(f"Training data shape: {X_train.shape}")
print(f"Validation data shape: {X_val.shape}")


Training data shape: (15200, 128, 128, 3)
Validation data shape: (3800, 128, 128, 3)


In [17]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Create an ImageDataGenerator for data augmentation
datagen = ImageDataGenerator(
    rotation_range=30,        # Random rotations up to 30 degrees
    width_shift_range=0.2,    # Randomly shift images horizontally
    height_shift_range=0.2,   # Randomly shift images vertically
    shear_range=0.2,          # Shear transformation
    zoom_range=0.2,           # Random zoom
    horizontal_flip=True,     # Random horizontal flip
    fill_mode='nearest'       # Fill in any missing pixels after transformations
)

# Fit the generator to your training data (X_train)
datagen.fit(X_train)

# Use the generator in model training
history = model.fit(
    datagen.flow(X_train, y_train, batch_size=32),  # Use the augmented data generator
    epochs=10,
    validation_data=(X_val, y_val),
    verbose=2
)

# Save the trained model
# model.save('quick_draw_model.h5')

# Evaluate the model on the validation set
val_loss, val_accuracy = model.evaluate(X_val, y_val, verbose=2)
print(f"Validation Loss: {val_loss}")
print(f"Validation Accuracy: {val_accuracy}")

Epoch 1/10


KeyboardInterrupt: 

In [ ]:
# Plot training vs validation accuracy/loss
import matplotlib.pyplot as plt

# Plot accuracy
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.show()

# Plot loss
plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label = 'val_loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc='upper right')
plt.show()

In [7]:
model.compile(optimizer=Adam(learning_rate=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])

# Continue training
history_finetune = model.fit(
    X_train, 
    y_train, 
    epochs=5,  # Fine-tune for fewer epochs
    batch_size=32,
    validation_data=(X_val, y_val),
    verbose=2
)

# Save the fine-tuned model
model.save('quick_draw_finetuned_model.h5')


Epoch 1/5
475/475 - 19s - 41ms/step - accuracy: 0.1053 - loss: 2.8717 - val_accuracy: 0.1066 - val_loss: 2.8715
Epoch 2/5
475/475 - 12s - 26ms/step - accuracy: 0.1051 - loss: 2.8725 - val_accuracy: 0.1066 - val_loss: 2.8714
Epoch 3/5
475/475 - 13s - 26ms/step - accuracy: 0.1055 - loss: 2.8716 - val_accuracy: 0.1066 - val_loss: 2.8714
Epoch 4/5
475/475 - 12s - 26ms/step - accuracy: 0.1052 - loss: 2.8722 - val_accuracy: 0.1066 - val_loss: 2.8714
Epoch 5/5
475/475 - 13s - 26ms/step - accuracy: 0.1052 - loss: 2.8716 - val_accuracy: 0.1066 - val_loss: 2.8713
